In [ ]:
# %pip install pandas, matplotlib, tqdm, neo4j, numpy

Downloaded Neo4jDesktop - 1.6.1 and created a Local Database 

In [7]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "2024-2025"))

In [ ]:
import pandas as pd

actions_df = pd.read_csv(
    "data/act-mooc/mooc_actions.tsv",
    sep="\t",
    header=None,
    names=["user_id", "target_id", "action_id", "timestamp"],
    dtype=str,
    low_memory=False
)

features_df = pd.read_csv(
    "data/act-mooc/mooc_action_features.tsv",
    sep="\t",
    header=None,
    names=["feature1", "feature2", "feature3", "feature4"],
    dtype=str,
    low_memory=False
)

labels_df = pd.read_csv(
    "data/act-mooc/mooc_action_labels.tsv",
    sep="\t",
    header=None,
    names=["label"],
    dtype=str,
    low_memory=False
)


# Reset the index to ensure alignment
actions_df.reset_index(drop=True, inplace=True)
features_df.reset_index(drop=True, inplace=True)
labels_df.reset_index(drop=True, inplace=True)

# Combine dataframes
combined_df = pd.concat([actions_df, features_df, labels_df], axis=1)

In [10]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411750 entries, 0 to 411749
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    411750 non-null  object
 1   target_id  411750 non-null  object
 2   action_id  411750 non-null  object
 3   timestamp  411750 non-null  object
 4   feature1   411750 non-null  object
 5   feature2   411750 non-null  object
 6   feature3   411750 non-null  object
 7   feature4   411750 non-null  object
 8   label      411750 non-null  object
dtypes: object(9)
memory usage: 28.3+ MB


In [ ]:
def insert_action(tx, user_id, target_id, action_id, timestamp,
                  feature1, feature2, feature3, feature4, label): # 9 x 30 = 270/60 = 4.5 hours
    tx.run("""
        MERGE (u:User {id: $user_id})
        MERGE (t:Target {id: $target_id})
        CREATE (u)-[:PERFORMS {
            action_id: $action_id,
            timestamp: $timestamp,
            feature1: $feature1,
            feature2: $feature2,
            feature3: $feature3,
            feature4: $feature4,
            label: $label
        }]->(t)
    """, user_id=user_id, target_id=target_id, action_id=action_id,
         timestamp=timestamp, feature1=feature1, feature2=feature2,
         feature3=feature3, feature4=feature4, label=label)

with driver.session() as session:
    for idx, row in combined_df.iterrows():
        session.execute_write(
            insert_action,
            row['user_id'], row['target_id'], row['action_id'], row['timestamp'],
            row['feature1'], row['feature2'], row['feature3'], row['feature4'], row['label']
        )
        if idx % 50000 == 0 and idx !=0 :
            print(f"Inserted {idx} rows...")

- MAYBE THIS WILL SPEED IT UP

In [ ]:
# def insert_actions_batch(tx, batch):
#     tx.run("""
#         UNWIND $rows AS row
#         MERGE (u:User {id: row.user_id})
#         MERGE (t:Target {id: row.target_id})
#         CREATE (u)-[:PERFORMS {
#             action_id: row.action_id,
#             timestamp: row.timestamp,
#             feature1: row.feature1,
#             feature2: row.feature2,
#             feature3: row.feature3,
#             feature4: row.feature4,
#             label: row.label
#         }]->(t)
#     """, rows=batch)

# # Set batch size
# batch_size = 1000

# with driver.session() as session:
#     for i in range(0, len(combined_df), batch_size):
#         batch_df = combined_df.iloc[i:i+batch_size]
#         batch = batch_df.to_dict('records')  # fast conversion
#         session.execute_write(insert_actions_batch, batch)
#         print(f"✅ Inserted {i + len(batch)} rows")

In [ ]:
def run_query(tx, query):
    result = tx.run(query)
    for record in result:
        print(record)

### CYPHER QUERIES EXECUTION

In [ ]:
with driver.session() as session:
    session.execute_read(run_query)